In [1]:
#%matplotlib notebook
from numpy import *
import bqplot.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Latex, HTML, Math
from scipy.constants import centi, angstrom, c, mega
from scipy.signal import convolve

In [25]:
# Doppler line shape in terms of FWHM, DnD
T = 300                 # kelvins
M = 23                  # g/mol for Na
nu_0 = centi / (5890*angstrom)  # wavelength = 5890 angstroms
DnD = 7.1e-7 * nu_0 * sqrt(T/M)

def g_D(nu, nu0 = nu_0, width = DnD):
    const = 2*sqrt(log(2)/pi)/width
    output = const * exp(-4*log(2)*((nu-nu0)/width)**2)
    return output / max(output)

def lorentz(nu, nu0 = nu_0, width = 300*mega*centi/c):
    output = width/(2*pi)/((width*0.5)**2 + (nu-nu0)**2)
    return output / max(output)

## Gaussian vs Lorentzian vs Voigt lineshapes

#### Gaussian line shape

Expressed in terms of the Full Width at Half Maximum (FWHM), $\Delta \nu_G$:

$$ g_G(\nu - \nu_0) = \frac{2}{\Delta \nu_G} \left( \frac{\ln{2}}{\pi } \right)^{1/2} e^{-4 \ln{2} [(\nu-\nu_0)/\Delta \nu_G]^2}$$

#### Lorentzian line shape

Expressed in terms of FWHM, $\Delta \nu_L$:

$$ g_L(\nu - \nu_0) = \frac{\Delta \nu_L / 2 \pi} {\left( \Delta \nu_L  / 2 \right)^2 + (\nu - \nu_0)^2}$$

#### Voigt line shape

This is a convolution of a Gaussian line shape with a Lorentzian line shape

$$g_V(\nu - \nu_0) = \int_{-\infty}^{\infty} g_G(\nu_0'-\nu_0) g_L(\nu - \nu_0') d\nu_0'$$

In the plot below, the line shapes are automatically normalized to a maximum of 1 (vertical scale, not integral) just to make them easily visible on the same scale.

In [30]:
# set up widgets
fig_layout = widgets.Layout(height = "400px", width = "600px")
fig_margin_dict = dict(top=30, bottom = 30, left = 20, right = 10)
figure = plt.figure(title = "Line Shape Comparison", layout = fig_layout, fig_margin = fig_margin_dict)

center = 0.0
gauss_width = 1300 # MHz
lorentz_width = 100 # MHz
max_width = max(gauss_width, lorentz_width)
max_width *= mega*centi/c
nu = linspace(-4*max_width, 4*max_width,  200)
y1 = g_D(nu, nu0 = center, width = gauss_width*mega*centi/c)
y1 = y1 / max(y1)
y2 = lorentz(nu, nu0 = center, width = lorentz_width*mega*centi/c)
y2 = y2 / max(y2)
result = convolve(y1, y2, mode='same') / sum(y2)
result = result / max(result)
shift = nu[argmax(y1)] - nu[argmax(result)]

gauss_line = plt.plot(nu, y1, labels = ["Gaussian"], colors = ['blue'], display_legend=True)
lorentz_line = plt.plot(nu, y2, labels = ["Lorentzian"], colors = ['magenta'], display_legend=True)
voigt_line = plt.plot(nu+shift, result, labels = ["Voigt"], colors = ['green'], display_legend=True)
#plt.grid()
plt.legend();

# rows of controls
# first row: Gaussian
label1 = widgets.Label("Gaussian width: ")
gaussSlider = widgets.FloatSlider(value = gauss_width, min = 1, max = 4000, step = 1)
unitLabel = widgets.Label("MHz")
row1 = widgets.HBox([label1, gaussSlider, unitLabel])

# second row:  Lorentzian
label2 = widgets.Label("Lorentzian width: ")
lorentzSlider = widgets.FloatSlider(value = lorentz_width, min = 1, max = 4000, step = 1)
row2 = widgets.HBox([label2, lorentzSlider, unitLabel])

# callbacks
def update_plot(change):
    g_width = gaussSlider.value *mega*centi/c
    l_width = lorentzSlider.value * mega*centi/c
    
    gauss_line.y = g_D(nu, nu0 = center, width = g_width)
    lorentz_line.y = lorentz(nu, nu0 = center, width = l_width)
    result = convolve(gauss_line.y, lorentz_line.y, mode='same')
    voigt_line.y = result / max(result)
    
gaussSlider.observe(update_plot, names = 'value')
lorentzSlider.observe(update_plot, names = 'value')
widgets.VBox([figure, row1, row2])

In [31]:
%%html
<style>
div.input{
    display:none;
}
</style>